# Coursera Project On Toronto Neighborhood Segmentation

### Will showcase web-scraping and geo-spatial clustering

In [12]:
#importing libraries that will be used
import pandas as pd #data analysis
#ensure that we can print out an entire dataset if desired
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import requests #handle web requests
from bs4 import BeautifulSoup #parse the resultant html

In [13]:
wikiurl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #link containing our data to be analyzed
table_class = "wikitable sortable jquery-tablesorter" #define type of table expected to be collected from target link
response = requests.get(wikiurl) #retrieve html data from target link
print(response.status_code) #HTML status code, 200 is expected as that indicates succescful retrieval

200


In [14]:
#convert the html data into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
#specify table object from html response
toronto_table=soup.find('table', {'class':"wikitable"})
toronto_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

From the output of the previous cell, we can see that the table is being located with the desired Toronto neighborhood data.

In [15]:
#convert BeautifulSoup object to Pandas DataFrame 
df_toronto = pd.read_html(str(toronto_table)) #using Pandas built in function for reading html
df_toronto = pd.DataFrame(df_toronto[0]) #coerce list into DataFrame
print(df_toronto.head()) #verify

  Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


The output of the cell above demonstrates that the table contains the features that we are looking for (Postal Code, Borough, Neighborhood), however it contains several instances on "Not assigned" data. For this, we will need to clean the data.

Since our goal is to segment Toronto neighborhoods, entry points with no neighborhoods are not useful to us; however, we can substitue the borough in cases where a neighborhood is not assigned. If no borough is available either, it will be necessary to drop the entry.

In [16]:
#Begin cleaning data
NoBoroughIndexes = df_toronto[df_toronto['Borough'] == 'Not assigned'].index #establish indexes where no borough is assigned
df_toronto_bor = df_toronto.drop(NoBoroughIndexes) #drop the indexes obtained
df_toronto_bor.head() #verify succesful drop

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In the output above, we can see that we have succesfully removed rows with no borough assigned, however,we should check to see if there is any data with a borough but no neighborhood.

In [17]:
print(df_toronto_bor[df_toronto_bor['Neighbourhood'] == 'Not assigned'])

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


We can see above that parsing the neighborhood column for 'Not assigned' entries returns nothing. However, we can double check to make sure there are no rows with unassigned neighborhoods under different entries.

In [18]:
df_neighborhoods = pd.DataFrame(df_toronto_bor['Neighbourhood'].value_counts())
df_neighborhoods

,Neighbourhood
Downsview,4
Don Mills,2
"Runnymede, Swansea",1
"Parkview Hill, Woodbine Gardens",1
Canada Post Gateway Processing Centre,1
Woburn,1
"Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",1
Woodbine Heights,1
St. James Town,1
"North Park, Maple Leaf Park, Upwood Park",1


We can see when looking at the counts for how many times a neighborhood shows up, there are no entries without assigned neighborhoods.
We can also see that many postal areas contain multiple nighborhoods, where neighboroods are separated by commas in the above table.

In [19]:
shape = df_toronto_bor.shape
row = shape[0]
col = shape[1]
fmt_str = "There are %i rows and %i columns in our cleaned data set" % (row, col)
print(fmt_str)

There are 103 rows and 3 columns in our cleaned data set


# Part Two:
## In this section (which will be saved as a new and separate notebook, as per the assignment's instructions), we will find the latitude and longitude data of the previously identified boroughs and neighborhoods

In [23]:
!pip install geocoder

     |████████████████████████████████| 98 kB 10.1 MB/s eta 0:00:01


In [24]:
#To do this, we are going to use geocoder
import geocoder

#we will start with an empty lat/long column
df_toronto_bor['Lat/Long'] = None
df_toronto_bor.reset_index(inplace=True)
df_toronto_bor.drop(['level_0', 'index'], axis=1, inplace=True)

KeyError: "['level_0'] not found in axis"

In [25]:
df_toronto_bor.head()

,index,Postal Code,Borough,Neighbourhood,Lat/Long
0,2,M3A,North York,Parkwoods,None
1,3,M4A,North York,Victoria Village,None
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",None
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",None
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",None


In [29]:
#We can then construct a loop to populate each postal code with a latitude and longitude value

#lat_lng_coords = None

#for code, data in (df_toronto_bor['Postal Code'].items()):
#    while(df_toronto_bor.loc[code, 'Lat/Long'] is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(data))
#        df_toronto_bor.loc[0, 'Lat/Long'] = g.latlng


KeyboardInterrupt: 

In [30]:
print(lat_lng_coords)
print(g)

None
<[REQUEST_DENIED] Google - Geocode [empty]>


Here we can see that the Geocode library is producing no results, as indicated by the instructions for this project. For this, the loop that would be used to generate the lat/long for each postal code has been commented out.

We will proceed using the CSV file of lat/long values instead

In [31]:
#We will have to import a library, io, to decode the CSV at the link provided
import io
csv_url = 'https://cocl.us/Geospatial_data' #Link to CSV
csv_response = requests.get(csv_url).content #Retrieve content at the link
ll_df = pd.read_csv(io.StringIO(csv_response.decode('utf-8'))) # Decode the content string and coerce it to df
ll_df.head() #Preview df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
#drop lat/long column made before failed loop 
df_toronto_bor.drop(['Lat/Long'], axis=1, inplace=True)
#merge original table with lat/long table
neighborhood_ll = pd.merge(df_toronto_bor, ll_df, on=['Postal Code', 'Postal Code'])
neighborhood_ll.drop(['index'], axis=1, inplace=True)
neighborhood_ll.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Here, we can see that the provided CSV has been uploaded and merged into the main dataframe, so now we have the postal code, corresponding borough, neighborhood, and latitude/longitude values for each.

# Part Three:

## In this section (which will also be saved/published as a new/separate notebook) we will visualize, segment, and cluster the neighborhood data

In [37]:
#import library for clustering
from sklearn.cluster import KMeans
#import library for visualizing geospatial data
import folium

In [48]:
#instantiate a map of the Toronto area with folium
map_toronto = folium.Map(location = [43.66, -79.38], zoom_start = 11) #Using the lat/long values of DT Toronto
map_toronto

In [49]:
#add markers for each neighbnorhood and borough, and redraw the map
for lat, lng, borough, neigh in zip(neighborhood_ll['Latitude'], neighborhood_ll['Longitude'],
                                   neighborhood_ll['Borough'], neighborhood_ll['Neighbourhood']):
    label = '{}, {}'.format(neigh,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
    
map_toronto

### Now that we have a labeled map of the Toronto area we can implement Foursquare's API to segment and cluster the area

In [50]:
#Components of the FourSquare API URL
CLIENT_ID='Y2CCZIUQQWKTENZRWWFBDQY3DNOSP3AYUTNDRNY3HYQZ3EV4' #Foursquare ID
CLIENT_SECRET='ZRRPIVBOFR2WTM4XGL0A4OTTGRFGMC0RZJ5QITOMG33UYWFL' #Foursquare Access Secret
VERSION='20180605' #Foursquare API Version
LIMIT=100 #Default Limit

Once we have our credentials ready to pass into the API, we can define a function that will pull those credentials into a URL to access the API. We can then call that function on the neighborhoods in our dataframe, and return the venues in those areas


In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500): #Begin function definition, specifying inputs
    
    venues_list=[] #Instantiate empty list to pass venues into
    #Open for loop to pass the name and location of each neighborhood into Foursquare's API
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name) #Progress Tracking that will print each neighborhood as it is processed
        
        #Create the API request URL skeleton
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius,
        LIMIT)

        #With the URL skeleton established, we can make a GET request that accepts it
        results = requests.get(url).json()["response"]['groups'][0]['items'] #from out lessons, we know this is the location of the data that we are looking for
        
        #We only want some of the information that is produced, and we can define that here
        venues_list.append([(
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                    'Neighborhood Latitude',
                    'Neighborhood Longitude',
                    'Venue',
                    'Venue Latitude', 
                    'Venue Longitude',
                    'Venue Category']
    
    return(nearby_venues)

In [53]:
#We can now psas our neighborhood data into this function and generate a new df for the resulting venues
toronto_venues = getNearbyVenues(names=neighborhood_ll['Neighbourhood'],
                                latitudes=neighborhood_ll['Latitude'],
                                longitudes=neighborhood_ll['Longitude']
                                )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

Lets check the size of the new dataframe, and preview the first five rows

In [54]:
print(toronto_venues.shape)
toronto_venues.head()

(2141, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


We can also look at the number of venues returned for each neighborhood

In [55]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


We can transform out data now to a more informative format: we can look at the types of venues present in each neighborhood. To do this, we can encode the venue category, group each neighborhood together, and analyze the average frequency of venue type within each neighborhood.

In [59]:
#We can transform the venue categories into columns of the table. Each row is an entry, and each value is a
#binary result signifying that a category of venue is present at that entry. 
#At this stage, each neighborhood will have one entry per venue, so this is a temporary stage
tor_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
print(tor_onehot.shape)
tor_onehot.head()

(2141, 273)


,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

We can see that in the dataframe above, each row is a particular venue in a particular neighborhood, however, we have lost some information, such as which neighborhood the row represents, so, we can add the neighborhood column back and place it at the beginning of the dataframe

In [63]:
#add neighborhood column
tor_onehot['Neighborhood'] = toronto_venues['Neighborhood']
#move neighborhood column to first position
col_name = 'Neighborhood'
first_col = tor_onehot.pop(col_name)
tor_onehot.insert(0,col_name, first_col)
tor_onehot.head()

,Neighborhood,Women's Store,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

It is not particularly informative to have a row for every venue in a neighborhood, so we will group the table by neighborhood and take the average frequency of each venue category. This will give a more general picture of what types of venues appear in which neighborhoods

In [64]:
tor_grp = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grp

,Neighborhood,Women's Store,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.00

In [65]:
tor_grp.shape

(96, 273)

This demonstrates that we have 96 neighborhoods with 273 venue types. We can also see that it is often the case that a venue does not show up in a neighborhood at all, or very seldom, so it may be more informative to generate the top five frequent venues for each neighborhood.

In [66]:
num_top_venues = 5

for hood in tor_grp['Neighborhood']:
    print("---"+hood+"---")
    temp = tor_grp[tor_grp['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp=temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp=temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---Agincourt---
                       venue  freq
0             Clothing Store   0.2
1                     Lounge   0.2
2               Skating Rink   0.2
3             Breakfast Spot   0.2
4  Latin American Restaurant   0.2


---Alderwood, Long Branch---
            venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2     Coffee Shop  0.14
3             Pub  0.14
4  Sandwich Place  0.14


---Bathurst Manor, Wilson Heights, Downsview North---
           venue  freq
0           Bank  0.10
1    Coffee Shop  0.10
2    Gas Station  0.05
3          Diner  0.05
4  Shopping Mall  0.05


---Bayview Village---
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4  Monument / Landmark  0.00


---Bedford Park, Lawrence Manor East---
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2      Sandwich Place  0.09
3   Indian Restaurant  0.05
4           Locksmith  0.

Here we can see a visual overview of the top five most frequent venue types within a neighborhood, however, we cannot manipulate this data until we store it in a dataframe. So, we should sort them in descending order, and then place the data into a new df.

In [70]:
#import library to return evenly spaced values within interval
import numpy as np
#it will be necessary to define a new function specific to the data we want to sort
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd'] #unique indicators for 1st, 2nd, 3rd
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) #if 1st, 2nd, 3rd
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) #anything greater than 3rd
        
#instantiate new df
n_venues_sorted = pd.DataFrame(columns=columns)
n_venues_sorted['Neighborhood'] = tor_grp['Neighborhood']

#loop for each neighborhood in grouped neighborhood df to be passed through most common venue function
for ind in np.arange(tor_grp.shape[0]):
    n_venues_sorted.iloc[ind,1:] = return_most_common_venues(tor_grp.iloc[ind, :], num_top_venues)
    
n_venues_sorted.head()                                                       

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Pharmacy,Pub
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Ice Cream Shop,Frozen Yogurt Shop,Sandwich Place
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Dessert Shop
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,American Restaurant,Pub


# Clustering
### Now that the neighborhoods can be characterized by their most common venues, we can perform k-means clustering on the neighborhoods

In [71]:
#we will set the number of clusters arbitrarily to 5
kclusters = 5

tor_grp_cls = tor_grp.drop('Neighborhood',1)
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(tor_grp_cls)

#check labels
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [102]:
#We can import the cluster identification into our dataframe with most common venues
n_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#Create new df to aggreate all data
tor_merge = neighborhood_ll
#rename to match spellings across df
tor_merge.rename({'Neighbourhood':'Neighborhood'}, axis=1, inplace=True)

#merge df's on the neighborhoods
tor_merge = tor_merge.join(n_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
tor_merge.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Food & Drink Shop,Park,Wings Joint,Dim Sum Restaurant,Diner
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Intersection,French Restaurant,Pizza Place,Coffee Shop,Hockey Arena
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Park,Pub,Bakery,Theater
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Women's Store,Furniture / Home Store,Vietnamese Restaurant,Accessories Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Art Gallery,Italian Restaurant,Beer Bar,Bar
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store
7,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Gym,Japanese Restaurant,Coffee Shop,Beer Store,Dim Sum Restaurant
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Pizza Place,Breakfast Spot,Gym / Fitness Center,Pet Store,Pharmacy
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant


We can see from the preview of the new dataframe that some neighborhoods were not clustered because they had no
venues present. For the purposes of this project, we can drop those rows from the table.

In [103]:
#drop rows with NA values
tor_merge.dropna(axis=0,inplace=True)

### With the top five venues per neighborhood identifed, each neighborhood assigned to a cluster, and the lat/long of each neighborhood provided, we can visualize the result of our clustering

In [105]:
tor_merge.head(10)
print(tor_merge.dtypes)

Postal Code               object
Borough                   object
Neighborhood              object
Latitude                 float64
Longitude                float64
Cluster Labels           float64
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
dtype: object


In [106]:
#Coerce Cluster Lables to int
tor_merge['Cluster Labels'] = tor_merge['Cluster Labels'].astype(int)
print(tor_merge.dtypes)

Postal Code               object
Borough                   object
Neighborhood              object
Latitude                 float64
Longitude                float64
Cluster Labels             int64
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
dtype: object


In [108]:
#create map
map_cls = folium.Map(location=[43.66, -79.38], zoom_start = 11)

#set cluster colors
import matplotlib.cm as cm
import matplotlib.colors as colors
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merge['Latitude'], tor_merge['Longitude'], tor_merge['Neighborhood'], tor_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=0.7).add_to(map_cls)
    
map_cls

We can now compare this map to the characteristics of each cluster

# Cluster 1

In [122]:
tor_merge.drop(['Borough', 'Postal Code'], axis=1, inplace = True)
tor_merge.loc[tor_merge['Cluster Labels'] == 0, tor_merge.columns[[0] +list(range(4,tor_merge.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Parkwoods,Food & Drink Shop,Park,Wings Joint,Dim Sum Restaurant,Diner
21,Caledonia-Fairbanks,Park,Pool,Women's Store,Grocery Store,Dance Studio
35,"East Toronto, Broadview North (Old East York)",Park,Intersection,Convenience Store,Escape Room,Electronics Store
49,"North Park, Maple Leaf Park, Upwood Park",Park,Bakery,Construction & Landscaping,Donut Shop,Diner
61,Lawrence Park,Park,Bus Line,Swim School,Escape Room,Electronics Store
64,Weston,Park,Wings Joint,Donut Shop,Dim Sum Restaurant,Diner
66,York Mills West,Convenience Store,Park,Donut Shop,Dim Sum Restaurant,Diner
85,"Milliken, Agincourt North, Steeles East, L'Amo...",Intersection,Park,Playground,Bakery,Doner Restaurant
91,Rosedale,Park,Playground,Trail,Doner Restaurant,Dessert Shop


# Cluster 2

In [123]:
tor_merge.loc[tor_merge['Cluster Labels'] == 1, tor_merge.columns[[0] +list(range(4,tor_merge.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Victoria Village,Intersection,French Restaurant,Pizza Place,Coffee Shop,Hockey Arena
2,"Regent Park, Harbourfront",Coffee Shop,Park,Pub,Bakery,Theater
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Women's Store,Furniture / Home Store,Vietnamese Restaurant,Accessories Store
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Art Gallery,Italian Restaurant,Beer Bar,Bar
7,Don Mills,Gym,Japanese Restaurant,Coffee Shop,Beer Store,Dim Sum Restaurant
8,"Parkview Hill, Woodbine Gardens",Pizza Place,Breakfast Spot,Gym / Fitness Center,Pet Store,Pharmacy
9,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant
10,Glencairn,Park,Pizza Place,Pub,Bakery,Japanese Restaurant
12,"Rouge Hill, Port Union, Highland Creek",Bar,Construction & Landscaping,Wings Joint,Discount Store,Distribution Center
13,Don Mills,Gym,Japanese Restaurant,Coffee Shop,Beer Store,Dim Sum Restaurant


# Cluster 3

In [124]:
tor_merge.loc[tor_merge['Cluster Labels'] == 2, tor_merge.columns[[0] +list(range(4,tor_merge.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
45,"York Mills, Silver Hills",Martial Arts School,Wings Joint,Donut Shop,Dim Sum Restaurant,Diner


# Cluster 4

In [125]:
tor_merge.loc[tor_merge['Cluster Labels'] == 3, tor_merge.columns[[0] +list(range(4,tor_merge.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,"West Deane Park, Princess Gardens, Martin Grov...",Print Shop,Wings Joint,Doner Restaurant,Dim Sum Restaurant,Diner


# Cluster 5

In [126]:
tor_merge.loc[tor_merge['Cluster Labels'] == 4, tor_merge.columns[[0] +list(range(4,tor_merge.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,"Malvern, Rouge",Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store


# We can see, with two very large clusters and three very small clusters, that our K may not have been an optimal selection. It is likely that we could derive more clusters from cluster 1 and 2, however, the project demonstrates effective clustering and geospatial visualize regardless.